# Programme de Transposition des notes Moodle vers SynapseS

> 
>1. Télécharger les fichiers Excel (de Moodle) et CSV de SynapseS
>2. Exécuter le programme
> 

In [ ]:
import pandas as pd
import re
import os
import csv
from IPython.display import FileLink, FileLinks
from IPython.core.display import display, HTML

def LetterToNum(letter,mode='TP'):
    if mode == 'X':
        # -- X --    
        if letter == 'A+':
            note = 19
        elif letter == 'A':
            note = 17
        elif letter == 'A-':
            note = 15
        elif letter == 'B+':
            note = 14
        elif letter == 'B':
            note = 13
        elif letter == 'B-':
            note = 12
        elif letter == 'C':
            note = 11
        elif letter == 'D':
            note = 9
        elif letter == 'E':
            note = 7
        elif letter == 'F':
            note = 0
        else:
            note = None
    else:
        # -- Telecom --
        if letter == 'A+':
            note = 17.1
        elif letter == 'A':
            note = 16.9
        elif letter == 'A-':
            note = 13.9
        elif letter == 'B+':
            note = 13.9
        elif letter == 'B':
            note = 12.9
        elif letter == 'B-':
            note = 10.9
        elif letter == 'C':
            note = 9.9
        elif letter == 'D':
            note = 7.9
        elif letter == 'E':
            note = 5
        elif letter == 'F':
            note = 0
        else:
            note = None
    return note

#MoodleFile = 'Copie de LAB103-2021 Grades_Moodle-1.xlsx'
#SynapseSFile = 'Fichier Synapses LAB103 vierge.csv'

display(HTML('<b>Saisir le nom du fichier de note Moodle (.xlsx):</b>'))
MoodleFile = input()
display(HTML('<b>Saisir le nom du fichier Synapses (.csv):</b>'))
SynapseSFile = input()

dfMoodle = pd.read_excel(MoodleFile) 
dfSynapses = pd.read_csv(SynapseSFile,delimiter=';', encoding='utf-8-sig',skiprows=1, on_bad_lines='skip')

s1 = dfSynapses['Matricule']
s2 = dfMoodle['Institution']
intersection = pd.Series(list(set(s1) & set(s2)))

display(HTML("<br>Les éléments suivants n'étaient pas dans Moodle"))

display(dfMoodle[['Institution','First name','Surname','Email address']][~dfMoodle['Institution'].isin(intersection)])

for e in intersection:
    dfSynapses.at[dfSynapses['Matricule'] == e,'Note initiale'] = LetterToNum(dfMoodle.loc[dfMoodle['Institution'] == e]['Course letter grade (Real)'].values[0])
   
# on écrit le csv
dfSynapses.to_csv('out.csv',sep=';', encoding='utf-8', index=False)

# on lit le fichier original pour récupérer la première ligne
with open("Fichier Synapses LAB103 vierge.csv",'r', encoding='utf-8-sig') as original:
    lines = []
    for line in original:
        lines.append(line)

# On écrit le fichier csv temporaire
with open("out.csv",'r') as csv:
    csvlines = []
    for line in csv:
        csvlines.append(line)

# On récupère le nom du cours 
cours = lines[0].split(';')[0]        

# On fusionne les informations
with open (f'{cours}.csv', 'w', encoding='utf-8-sig') as fp:
    fp.write(lines[0])
    for l in csvlines:
        fp.write(l)

# on efface le fichier temporaire
os.remove('out.csv')

# on affiche le résultat
display(HTML(f'Le fichier resultat à télécharger est : {cours}.csv'),FileLink(f'{cours}.csv'))
